# Install packages

In [34]:
%%capture --no-stderr
!pip install -q langchain_google_genai langchain_core langchain_community tavily-python pydantic

# Import packages

In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from pydantic import BaseModel

 # Set up authentication keys

In [37]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

In [38]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    api_key = GEMINI_API_KEY,
    temperature = 0)

In [39]:
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--aa36ff6c-22a2-473c-9d40-8f77fa5e3701-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [ ]:
messages = [
    HumanMessage(content="Hi", name="Human Student"),
    AIMessage(content='Hi! How can I help you today? \n', name="AI Assistant"),
    HumanMessage(content="What is LangChain?", name="Human Student"),
    AIMessage(content='LangChain is a framework for developing applications powered by language models.', name="AI Assistant"),
    HumanMessage(content="How can I learn", name="Human Student"),
    ]

response = llm.invoke(messages)
messages.append(response)

# for message in messages:
#   message.pretty_print()

# State

First, define the State of the graph.

The State schema serves as the input schema for all Nodes and Edges in the graph.

Each node returns a new value of the state key graph_state.

By default, the new value returned by each node will override the prior state value.

In [51]:
class learning_state(BaseModel):
  message : str

In [67]:
# prompt: create an example from above LearningState
state: learning_state = learning_state(message = "hello from world")
print(state)
print(state.message +" I am")
print(state)
print(type(state))

message='hello from world'
hello from world I am
message='hello from world'
<class '__main__.learning_state'>
